In [1]:
# loss  二次代价函数

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 载入数据集
mnist = input_data.read_data_sets('./', one_hot=True)


# 设置每个 batch_size
batch_size = 100
# 共需多少批次运算
batch_nums = mnist.train.num_examples // batch_size

# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

# 定义 placeholder
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')

with tf.name_scope('layer1'):
    # 神经网络层
    with tf.name_scope('weight1'):
        w1 = tf.Variable(tf.zeros([784, 10]), name='w1')
        variable_summaries(w1)
    with tf.name_scope('biases1'):
        b1 = tf.Variable(tf.zeros([10]), name='b1')
        variable_summaries(b1)
    # 更合理的 初始值设置
    # w1 = tf.Variable(tf.truncated_normal([784, 2000], stddev=0.1))
    # b1 = tf.Variable(tf.zeros([2000] + 0.1))
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(tf.matmul(x, w1) + b1)

with tf.name_scope('loss'):
    # loss function
    # ESM
    loss = tf.reduce_mean(tf.square(y-prediction))
    # CE 交叉熵
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=predictions))

with tf.name_scope('train'):
    # 梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init_op = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_predictions'):
        # 预测结果 存储在一列表中
        correct_predictions = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    
    with tf.name_scope('accuracy'):
        # 计算 准确率
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

with tf.Session() as sess:
    sess.run(init_op)
    writer = tf.summary.FileWriter('logs/', sess.graph)
    for epoch in range(1):
        for batch in range(batch_nums):
            batch_xs, batch_ys = mnist.train.next_batch(batch)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
            
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print('Iter epoch {0}, Testing accuracy is {1}'.format(epoch, acc))
    

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Iter epoch 0, Testing accuracy is 0.8303999900817871
